In [1]:
import geemap
import json
import os
import requests
import ee
from geemap import geojson_to_ee, ee_to_geojson

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin

from ipyleaflet import (Choropleth, Marker, Popup, WidgetControl)
from ipywidgets import (HTML, Text)

from matplotlib import pyplot as plt
from matplotlib import style
import matplotlib as plt
import csv



In [2]:
ee.Initialize()


In [3]:
df = pd.read_excel('Food_table_2013.xlsx')

In [4]:

USA_df = df[df['Area Abbreviation'] == 'USA']
UK_df = df[df['Area Abbreviation'] == 'GBR']
FRA_df = df[df['Area Abbreviation'] == 'FRA']

joined_df = USA_df.append([UK_df, FRA_df])

# reindex_df = joined_df.reset_index(drop=True)

# reindex_df[2012]
joined_df = joined_df[['Area Abbreviation', 'Area Code', 'Item', 'Item Code', 'Element', 'Element Code', 'Unit', 'latitude', 'longitude', 2012, 2013]]
joined_df
# columns = ['Area Abbreviation', 'Area Code', 'Item', 'Item Code', 'Element', 'Element Code', 'Unit', 'latitude', 'longitude', '2012', '2013']
# new_df = joined_df.reindex(columns=columns)
# new_df[columns] = new_df[columns]
# df2 = df.rename({'Area Abbreviation': 'Area Abbrevation', 'Area Code': 'Area Code', 'Item': 'Item', 'Item Code': 'Item Code', 'Element': 'Element', 'Element Code': 'Element Code', 'Unit': 'Unit', 'latitude': 'latitude', 'longitude': 'longitude', '2012': 'Twenty Twelve', '2013': 'Twenty Thirteen'}, axis=1)  # new method
# df2 = df.rename({'Area Abbreviation': 'Area Abbrevation', 'Area Code': 'Area Code', 'Item': 'Item', 'Item Code': 'Item Code', 'Element': 'Element', 'Element Code': 'Element Code', 'Unit': 'Unit', 'latitude': 'latitude', 'longitude': 'longitude', '2012': 'Twenty Twelve', '2013': 'Twenty Thirteen'}, axis= 'columns')
# df2 = df.rename(columns={'Area Abbreviation': 'Area Abbrevation', 'Area Code': 'Area Code', 'Item': 'Item', 'Item Code': 'Item Code', 'Element': 'Element', 'Element Code': 'Element Code', 'Unit': 'Unit', 'latitude': 'latitude', 'longitude': 'longitude', '2012': 'Twenty Twelve', '2013': 'Twenty Thirteen'})
# joined_df.append()


# joined_df[['Area Code', 'Item Code', '2013']]
# cleaned_joined_df = joined_df[['Area Abbreviation', 'Area Code', 'Item', 'Item Code', 'Element', 'Element Code', 'Unit', 'latitude', 'longitude']]

# cleaned_joined_df
# no_zeros = (cleaned_joined_df != 0).any(axis=1)

# new_zero_df = cleaned_joined_df.loc[no_zeros]
# new_zero_df

,Area Abbreviation,Area Code,Item,Item Code,Element,Element Code,Unit,latitude,longitude,2012,2013
20394,USA,231,Wheat and products,2511,Feed,5521,21393 tonnes,37.09,-95.71,9943,6196
20395,USA,231,Wheat and products,2511,Food,5142,21394 tonnes,37.09,-95.71,25757,25742
20396,USA,231,Rice (Milled Equivalent),2805,Food,5142,21395 tonnes,37.09,-95.71,2175,2203
20397,USA,231,Barley and products,2513,Feed,5521,21396 tonnes,37.09,-95.71,1441,1440
20398,USA,231,Barley and products,2513,Food,5142,21397 tonnes,37.09,-95.71,160,160
...,...,...,...,...,...,...,...,...,...,...,...
7045,FRA,68,Milk - Excluding Butter,2948,Food,5142,8044 tonnes,46.23,2.21,15151,15514
7046,FRA,68,"Fish, Seafood",2960,Feed,5521,8045 tonnes,46.23,2.21,115,129
7047,FRA,68,"Fish, Seafood",2960,Food,5142,8046 tonnes,46.23,2.21,2125,2152
7048,FRA,68,"Aquatic Products, Other",2961,Food,5142,8047 tonnes,46.23,2.21,5,4


In [5]:
df['longitude'] = df['longitude'].astype(int) 
df['latitude'] = df['latitude'].astype(int)

In [6]:
Map = geemap.Map()

# fc = geemap.pandas_to_ee(df, latitude="latitude", longitude="longitude")
# Map.addLayer(fc, "pandas to ee")
Map

# countries = geemap.shp_to_ee(countries_shp)
# Map.addLayer(countries, {}, 'Countries')

# df = geemap.ee_to_pandas(countries, selectors=['id', 'name'])
# df.head()



Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [27]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

map_df = df[['Area Abbreviation', 'Area Code', 'Item', 'Item Code', 'Element', 'Element Code', 'Unit', 'latitude', 'longitude', 2012, 2013]]

map_df

,Area Abbreviation,Area Code,Item,Item Code,Element,Element Code,Unit,latitude,longitude,2012,2013
0,AFG,2,Wheat and products,2511,Food,5142,1000 tonnes,33,67,4810,4895
1,AFG,2,Rice (Milled Equivalent),2805,Food,5142,1000 tonnes,33,67,425,422
2,AFG,2,Barley and products,2513,Feed,5521,1001 tonnes,33,67,367,360
3,AFG,2,Barley and products,2513,Food,5142,1002 tonnes,33,67,78,89
4,AFG,2,Maize and products,2514,Feed,5521,1003 tonnes,33,67,200,200
...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,181,Milk - Excluding Butter,2948,Food,5142,22471 tonnes,-19,29,426,451
21473,ZWE,181,"Fish, Seafood",2960,Feed,5521,22472 tonnes,-19,29,15,15
21474,ZWE,181,"Fish, Seafood",2960,Food,5142,22473 tonnes,-19,29,40,40
21475,ZWE,181,"Aquatic Products, Other",2961,Food,5142,22474 tonnes,-19,29,0,0


In [37]:
# {
#   "type": "Feature",
#   "geometry": {
#     "type": "Point",
#     "coordinates": [125.6, 10.1]
#   },
#   "properties": {
#     "name": "Dinagat Islands"
#   }
# }
country_food = {
  "type": "FeatureCollection", 
  "features": []
}

for row in map_df.values:
  country_food["features"].append({
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [row[7],row[8]]
  },
  "properties": { 
    "Area_Abbreviation": row[0], 
    "Area_Code": row[1],
    "Item": row[2],
    "Item_Code": row[3],
    "Element": row[4],
    "Element_Code": row[5],
    "Unit": row[6],
    "v2012": row[9],
    "v2013": row[10]
  }
  })

country_food



{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [33, 67]},
   'properties': {'Area_Abbreviation': 'AFG',
    'Area_Code': 2,
    'Item': 'Wheat and products',
    'Item_Code': 2511,
    'Element': 'Food',
    'Element_Code': 5142,
    'Unit': '1000 tonnes',
    'v2012': 4810,
    'v2013': 4895}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [33, 67]},
   'properties': {'Area_Abbreviation': 'AFG',
    'Area_Code': 2,
    'Item': 'Rice (Milled Equivalent)',
    'Item_Code': 2805,
    'Element': 'Food',
    'Element_Code': 5142,
    'Unit': '1000 tonnes',
    'v2012': 425,
    'v2013': 422}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [33, 67]},
   'properties': {'Area_Abbreviation': 'AFG',
    'Area_Code': 2,
    'Item': 'Barley and products',
    'Item_Code': 2513,
    'Element': 'Feed',
    'Element_Code': 5521,
    'Unit': '1001 tonnes',
    'v2012': 367,
    'v2013': 36

In [39]:
with open('country_foods.geojson','w')as open_file:
    open_file.write(str(country_food).replace("'", '"'))




In [18]:
map_df.dropna()

gdf = gpd.GeoDataFrame(
    map_df, geometry=gpd.points_from_xy(map_df['longitude'], map_df['latitude']))



,Area Abbreviation,Area Code,Item,Item Code,Element,Element Code,Unit,latitude,longitude,2012,2013,geometry
0,AFG,2,Wheat and products,2511,Food,5142,1000 tonnes,33,67,4810,4895,POINT (67.00000 33.00000)
1,AFG,2,Rice (Milled Equivalent),2805,Food,5142,1000 tonnes,33,67,425,422,POINT (67.00000 33.00000)
2,AFG,2,Barley and products,2513,Feed,5521,1001 tonnes,33,67,367,360,POINT (67.00000 33.00000)
3,AFG,2,Barley and products,2513,Food,5142,1002 tonnes,33,67,78,89,POINT (67.00000 33.00000)
4,AFG,2,Maize and products,2514,Feed,5521,1003 tonnes,33,67,200,200,POINT (67.00000 33.00000)
...,...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,181,Milk - Excluding Butter,2948,Food,5142,22471 tonnes,-19,29,426,451,POINT (29.00000 -19.00000)
21473,ZWE,181,"Fish, Seafood",2960,Feed,5521,22472 tonnes,-19,29,15,15,POINT (29.00000 -19.00000)
21474,ZWE,181,"Fish, Seafood",2960,Food,5142,22473 tonnes,-19,29,40,40,POINT (29.00000 -19.00000)
21475,ZWE,181,"Aquatic Products, Other",2961,Food,5142,22474 tonnes,-19,29,0,0,POINT (29.00000 -19.00000)


In [9]:
print(gdf.head())

  Area Abbreviation  Area Code                      Item  Item Code Element  \
0               AFG          2        Wheat and products       2511    Food   
1               AFG          2  Rice (Milled Equivalent)       2805    Food   
2               AFG          2       Barley and products       2513    Feed   
3               AFG          2       Barley and products       2513    Food   
4               AFG          2        Maize and products       2514    Feed   

   Element Code         Unit  latitude  longitude  2012  2013  \
0          5142  1000 tonnes        33         67  4810  4895   
1          5142  1000 tonnes        33         67   425   422   
2          5521  1001 tonnes        33         67   367   360   
3          5142  1002 tonnes        33         67    78    89   
4          5521  1003 tonnes        33         67   200   200   

                    geometry  
0  POINT (67.00000 33.00000)  
1  POINT (67.00000 33.00000)  
2  POINT (67.00000 33.00000)  
3  POINT (

In [26]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

ax = world[world.continent == 'World'].plot(color = 'white', edgecolor = 'black')

# world.plot(ax=ax, color='blue')
# gdf.plot(columns= 2012)

gdf



C:\Users\Samuel M Schappel\.conda\envs\gee\lib\site-packages\geopandas\plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

,Area Abbreviation,Area Code,Item,Item Code,Element,Element Code,Unit,latitude,longitude,2012,2013
0,AFG,2,Wheat and products,2511,Food,5142,1000 tonnes,33.94,67.71,4810,4895
1,AFG,2,Rice (Milled Equivalent),2805,Food,5142,1000 tonnes,33.94,67.71,425,422
2,AFG,2,Barley and products,2513,Feed,5521,1001 tonnes,33.94,67.71,367,360
3,AFG,2,Barley and products,2513,Food,5142,1002 tonnes,33.94,67.71,78,89
4,AFG,2,Maize and products,2514,Feed,5521,1003 tonnes,33.94,67.71,200,200
...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,181,Milk - Excluding Butter,2948,Food,5142,22471 tonnes,-19.02,29.15,426,451
21473,ZWE,181,"Fish, Seafood",2960,Feed,5521,22472 tonnes,-19.02,29.15,15,15
21474,ZWE,181,"Fish, Seafood",2960,Food,5142,22473 tonnes,-19.02,29.15,40,40
21475,ZWE,181,"Aquatic Products, Other",2961,Food,5142,22474 tonnes,-19.02,29.15,0,0


In [ ]:
counter = 1
for i in map_df:
    if map_df['Area Code'] == counter + 1:
        map_df['2012_sum'] == sum(map_df['2012'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().